In [1]:
import numpy as np
from keras.utils import np_utils
from keras.datasets import mnist

def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    number = 10000
    x_train = x_train[0:number]
    y_train = y_train[0:number]
    x_train = x_train.reshape(number, 28, 28, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    x_test = x_test.astype('float32')
    
    y_train = np_utils.to_categorical(y_train, 10)
    y_test = np_utils.to_categorical(y_test, 10)
    '''
    x_train_max = np.amax(x_train, 0)
    x_test_max = np.amax(x_test, 0)
    x_train = x_train / (x_train_max + 1) #255
    x_test = x_test / (x_test_max + 1) #255
    '''
    x_train = x_train / 255
    x_test = x_test / 255
    
    ratio = 1
    x_train = (x_train * ratio + np.random.normal(loc=0.5, scale=0.16, size=(number, 28, 28, 1))) / (ratio + 1)
    x_test = (x_test * ratio + np.random.normal(loc=0.5, scale=0.16, size=(x_test.shape[0], 28, 28, 1))) / (ratio + 1)
    
    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_data()
print('Shape of x_train: ', np.shape(x_train))
print('Shape of y_train: ', np.shape(y_train))
print('Shape of x_test: ', np.shape(x_test))
print('Shape of y_test: ', np.shape(y_test))

Using TensorFlow backend.


Shape of x_train:  (10000, 28, 28, 1)
Shape of y_train:  (10000, 10)
Shape of x_test:  (10000, 28, 28, 1)
Shape of y_test:  (10000, 10)


In [2]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Flatten
from keras.optimizers import SGD, Adam, Adadelta
import keras.callbacks
import argparse

# if K.backend() == 'tensorflow':
# tensorboard = TensorBoard(log_dir=os.path.join(save_path, 'logs'), histogram_freq=10, write_graph=True)
# callbacks.append(tensorboard)

model = Sequential()
model.add(Conv2D(64, (5, 5), input_shape=(28, 28, 1), padding='valid', data_format='channels_last',  activation='selu', kernel_initializer='lecun_normal'))
model.add(ZeroPadding2D(padding=(2,2), data_format = 'channels_Last'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3, 3), padding='valid', data_format='channels_last', activation='selu', kernel_initializer='lecun_normal'))
model.add(ZeroPadding2D(padding=(1,1), data_format = 'channels_Last'))

model.add(Conv2D(32, (3, 3), padding='valid', data_format='channels_last', activation='selu', kernel_initializer='lecun_normal'))
model.add(ZeroPadding2D(padding=(1,1), data_format = 'channels_Last'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(16, (3, 3), padding='valid', data_format='channels_last', activation='selu', kernel_initializer='lecun_normal'))

model.add(Flatten())

#model.add(Dense(input_dim = 28*28, units = 689, activation = 'relu'))

for i in range(1):
    model.add(Dense(units = 128, activation = 'selu', kernel_initializer='lecun_normal'))
    model.add(Dropout(0.5))

model.add(Dense(units = 10, activation = 'softmax', kernel_initializer='lecun_normal'))

opt = Adadelta(lr = 0.05, rho = 0.95, epsilon = 1e-06)
#opt = 'adam'
model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics = ['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 64)        1664      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 28, 28, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 32)        18464     
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 14, 14, 32)        0         
__________

In [4]:
import os

model.fit(x_train, y_train, batch_size = 100, epochs = 4)

model_name = 'MNIST_model_fixed_20180118-2'
base_dir = 'C:/Users/User/Raw data/CNN_test'
mdl_dir = os.path.join(base_dir, 'model')
if not os.path.exists(mdl_dir):
    os.makedirs(mdl_dir)
model_dir = os.path.join(mdl_dir, model_name)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
model_path = os.path.join(model_dir, 'model-%d.h5' %(epochs))
model.save(model_path)
model_weights_path = os.path.join(model_dir, 'model_weights-%d.h5' %(epochs))
model.save_weights(model_weights_path)

result = model.evaluate(x_test, y_test)

print('Test score:',result[0])
print('Test accuracy:',result[1])

Epoch 1/4
10000/10000 [==============================] - 1s - loss: 0.2419 - acc: 0.9275     
Epoch 2/4
10000/10000 [==============================] - 1s - loss: 0.2087 - acc: 0.9384     
Epoch 3/4
10000/10000 [==============================] - 1s - loss: 0.1874 - acc: 0.9449     
Epoch 4/4
10000/10000 [==============================] - 1s - loss: 0.1712 - acc: 0.9484     


NameError: name 'e' is not defined

In [6]:
import numpy as np
import argparse
from keras.utils.vis_utils import plot_model
from sklearn.metrics import confusion_matrix
from contextlib import contextmanager

@contextmanager
def printoptions(*args, **kwargs):
    original_options = np.get_printoptions()
    np.set_printoptions(*args, **kwargs)
    try:
        yield
    finally:
        np.set_printoptions(**original_options)

model.save('weight_filepath.HDF5')

predictions = model.predict(x_test)
predictions = predictions.argmax(axis=-1)
y_labels = y_test.argmax(axis=-1)
sum_true = np.sum(y_test, axis=0)
print('True: ', sum_true)
confusions = confusion_matrix(predictions, y_labels)
print('confusions [prediction\True]')
print(confusions)
print('True rate: ')
with printoptions(precision=3, suppress=True):
    print('{}'.format(confusions/sum_true))

result = model.evaluate(x_test, y_test)
print('Test score:',result[0])
print('Test accuracy:',result[1])

True:  [  980.  1135.  1032.  1010.   982.   892.   958.  1028.   974.  1009.]
confusions [prediction\True]
[[ 969    0    3    1    1    2    3    1    4    4]
 [   0 1128    4    0    2    0    4    8    0    4]
 [   1    3  999    3    2    0    0   19    6    0]
 [   0    2    5  988    1    6    0    5    4    3]
 [   0    0    0    0  955    0    2    1    1    9]
 [   0    0    1    5    0  877    4    0    0   13]
 [   5    1    1    0    3    2  943    0    1    0]
 [   1    0    5    4    2    2    0  982    3    7]
 [   4    1   14    8    4    3    2    2  952    9]
 [   0    0    0    1   12    0    0   10    3  960]]
True rate: 
[[ 0.989  0.     0.003  0.001  0.001  0.002  0.003  0.001  0.004  0.004]
 [ 0.     0.994  0.004  0.     0.002  0.     0.004  0.008  0.     0.004]
 [ 0.001  0.003  0.968  0.003  0.002  0.     0.     0.018  0.006  0.   ]
 [ 0.     0.002  0.005  0.978  0.001  0.007  0.     0.005  0.004  0.003]
 [ 0.     0.     0.     0.     0.973  0.     0.002  0.001

In [13]:
import tensorflow as tf
import os
from keras.backend.tensorflow_backend import set_session
#os.environ["CUDA_VISIBLE_DEVICES"] = "5,10"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))